# 各種 Import

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

c:\Users\pauls\anaconda3\envs\BERT_Practice\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 資料集介紹
[**IMDB Dataset of 50K Movie Reviews**](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)  
資料集描述：取一段電影評論 (review)，每筆評論會有對應的評價(sentiment)，表示好評或負評  
總資料數：50000  
好評、負評比 = 1 : 1  


## 資料集處理 (單純的 pandas 切分)
預計將 訓練 : 驗證 : 測試 切成 60 : 20 : 20  
配合[原始範例程式碼](https://gist.github.com/vincenttzc/ceaa4aca25e53cb8da195f07e7d0af92#file-trainer_train_predict-py)，這邊用 pandas 把資料分成兩份

In [5]:
df = pd.read_csv('IMDB Dataset.csv')

# 把 "sentiment" 標籤的 "positive"、"negative" 換成 "1"、"0"
for i in range(len(df['sentiment'])):
    if df.at[i, 'sentiment'] == 'positive':
        df.at[i, 'sentiment'] = 1
    else:
        df.at[i, 'sentiment'] = 0
 
# 先根據分類 (0、1) 分成兩群，以便切分後的以便切分後的訓練、測試集的正、反資料集保持相同
groups = df.groupby(df.sentiment)
data_positive = groups.get_group(1)
data_negative = groups.get_group(0)

# 用 sample 進行抽樣 (講白話就是隨機打亂)
data_positive = data_positive.sample(frac=1.0)
data_negative = data_negative.sample(frac=1.0)

# 用 index 去切分訓練、資料集，再各自將正、反資料結合並存取
test_positive = data_positive.iloc[20000:, :]
test_negative = data_negative.iloc[20000:, :]
test_data = pd.concat([test_positive, test_negative], axis = 0, ignore_index=True).sample(frac=1)
train_positive = data_positive.iloc[:20000, :]
train_negative = data_negative.iloc[:20000, :]
train_data = pd.concat([train_positive, train_negative], axis = 0, ignore_index=True).sample(frac=1)

test_data.to_csv('test.csv', index = None)
train_data.to_csv('train.csv', index = None)

## 資料前處理
1. BERT cased vs. BERT uncased  
   BERT cased: 字母保留原樣  
   BERT uncased: 在做 WordPiece tokenization 前，會先將所有字母變小寫 (lowercased)、附加符號 (Accent markers) 拔掉  
2. BertModel vs. BertForSequenceClassification  
   BertModel: 基本 BERT 模型  
   BertForSequenceClassification: 針對序列 (文本) 分類做過 Fine-tune 的 BERT 模型

In [3]:
# Read data
data = pd.read_csv("train.csv")

# Define pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # num_labels: 分類類別數量

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["review"])
y = list(data["sentiment"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)          # 用 sklearn 將 train data 切分成訓練、驗證資料集，test_size = 0.25 表示 test 佔 25%
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# 建立 Model
* [Train](https://huggingface.co/docs/transformers/training#train)
  * [```Trainer```](https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.Trainer)  
    一些常用參數如下：
    * ```model```: 使用的 Model
    * ```args```: 訓練的相關參數
    * ```train_dataset```: 訓練資料集
    * ```eval_dataset```: 驗證資料集
    * ```compute_metrics```: 設定評估指標
    * ```callbacks```: (資料型態：list) 設定訓練模型過程中的觸發事件
      > Early Stopping:  
      > 參考資料：[連結](https://clay-atlas.com/blog/2020/09/29/pytorch-cn-early-stopping-code/)  
      > 為了預防 Overfitting，我們可以嘗試在模型失去泛化性前就先停止訓練  
      > ```early_stopping_patience```: 設定 loss (accuracy) 連續變差幾次就停止訓練
  * [```TrainingArguments```](https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments)  
    一些常用的參數如下：  
    * ```output_dir```: 存放模型訓練期間可能產生的檔案 (如：checkpoint, log 等等) 的路徑
    * ```evaluation_strategy```: 做評估的頻率單位
      * ```"no"```: 不在訓練期間做評估
      * ```"steps"```: 訓練時，每過 ```eval_steps``` 做一次評估
      * ```epoch```: 每個 epoch 結束做一次評估
    * ```eval_steps```: 每隔多少 steps 做一次評估
    * ```per_device_train_batch_size```: 每個 GPU/TPU core/CPU 訓練的 batch size
    * ```per_device_eval_batch_size```: 每個 GPU/TPU core/CPU 評估的 batch size
    * ```num_train_epochs```: 訓練的 Epoch 數
    * ```seed```: Random seed (預設為 42)，以確保模型的再現性 (reproducibility)
    * ```load_best_model_at_end```: 訓練結束後是否載入最佳模型

In [4]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=16)],
)

# Train pre-trained model
trainer.train()

c:\Users\pauls\anaconda3\envs\BERT_Practice\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11250
  Number of trainable parameters = 109483778


  0%|          | 0/11250 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8


{'loss': 0.4278, 'learning_rate': 4.7777777777777784e-05, 'epoch': 0.13}


  0%|          | 0/1250 [00:00<?, ?it/s]

...

Saving model checkpoint to output\checkpoint-11000
Configuration saved in output\checkpoint-11000\config.json


{'eval_loss': 0.3215979337692261, 'eval_accuracy': 0.939, 'eval_precision': 0.9296905601253427, 'eval_recall': 0.949779911964786, 'eval_f1': 0.9396278701504354, 'eval_runtime': 152.0617, 'eval_samples_per_second': 65.763, 'eval_steps_per_second': 8.22, 'epoch': 2.93}


Model weights saved in output\checkpoint-11000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from output\checkpoint-3500 (score: 0.23376521468162537).


{'train_runtime': 7705.753, 'train_samples_per_second': 11.68, 'train_steps_per_second': 1.46, 'train_loss': 0.19825113050672744, 'epoch': 3.0}


TrainOutput(global_step=11250, training_loss=0.19825113050672744, metrics={'train_runtime': 7705.753, 'train_samples_per_second': 11.68, 'train_steps_per_second': 1.46, 'train_loss': 0.19825113050672744, 'epoch': 3.0})

# 預測

In [4]:
# ----- 3. Predict -----#
# Load test data
test_data = pd.read_csv("test.csv")
X_test = list(test_data["review"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "output/checkpoint-11000"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


  0%|          | 0/1250 [00:00<?, ?it/s]

In [6]:
y_ori = np.array(test_data["sentiment"])
cnt = 0
y_len = len(y_pred)

for i in range(y_len):
    cnt += 1 if (y_ori[i] == y_pred[i]) else 0

print("Accuracy: ", float(cnt / y_len))

Accuracy:  0.9734
